# 나무 위키 RAG를 위한 메타 데이터를 활용한 벡터 스토어 생성 데모 노트북

### FaissMetaVectorStore
- 가장 많이 쓰이는벡터 스토어인 `FAISS`를 상속받아 활용한 벡터스토어
- 기존 벡터데이터가 오로지 `Document`들의 `page_content`만 임베딩하는 것과 달리
- 사용자가 벡터스토어 생성시 `Document`들이 공통으로 가지고 있는 `metadata`들의 필드 중 특정 필드를 페이지 컨텐츠와 독립적으로 임베딩하여 저장함
    - `FAISS`가 `index`라는 벡터 저장소를 하나만 가지고 있는 것과 달리 `FaissMetaVectorStore`는 `index`, `metaindex` 두개를 가지고 있음
    - `index` : `page_content`를 임베딩한 것
    - `metaindex`: `metadata` 중 특정 key값들의 value들을 concat한 문자열을 임베딩해 갖고 있음
- 사용법 예시: `vs_meta = FaissMetaVectorStore.from_documents(docs, embedding = EMBEDDING_FUNC, metadata_fields= ["abs_page_toc_item", "base_page_url"])` => abs_page_toc_item와 base_page_url이 metadata에 공통으로 있는 key 들이어야 함

- as_retriever()로 리트리버를 생성 시 두 좋류의 리트리버를 생성 가능
    - `vs.as_retriever(vectorStoreType='page_content', search_kwargs={'k': k})`: 일반적인 페이지 컨텐츠 임베딩에 따른 리트리버
    - `vs.as_retriever(vectorStoreType='metadata', search_kwargs={'k': k})`: 위에서 설정한 메타 데이터 임베딩에 따른 리트리버
    - 두 리트리버를 `EnsembleRetriever`로 섞어 사용 가능 하다.


## 모델 테스트 준비

> 관련 패키지 import

In [1]:
import os
from operator import itemgetter
from time import time

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import ConfigurableField

from namu_loader import NamuLoader

from langchain_community.vectorstores.faiss import FAISS
from FaissMetaVectorStore import FaissMetaVectorStore # Custom 제작 메타 데이터 벡터스토어

> 경로, API 키 설정 등

In [2]:
cur_notebook_dir = os.getcwd()
os.chdir("../")
base_dir = os.getcwd()
key_dir = os.path.join(base_dir, 'keys')
os.chdir(cur_notebook_dir)
print(os.getcwd())

from dotenv import load_dotenv
print(load_dotenv(dotenv_path= os.path.join(key_dir, ".env")))
os.chdir(cur_notebook_dir)

EMBEDDING_FUNC = OpenAIEmbeddings()

c:\Users\thdwo\Documents\Github\proj_artist_info_gen
True


> 기타 체인, 답변 관련 함수 정의, LLM 모델 정의

In [3]:
def get_documnet_from_namuwiki(url, NamuLoader_obj, hop = 1):
    loader = NamuLoader_obj(url, hop, verbose = True)
    docs = loader.load()

    # splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=500,
    #     chunk_overlap=20
    #     )
    # docs = splitter.split_documents(docs)
    return docs

def get_chain(retriever, model):
    prompt = ChatPromptTemplate.from_template("""
        Answer the users's QUESTION based on the CONTEXT documents. Use the language that the user types in.
        If you can not find an appropriate answer, return "No Answer found.".
        CONTEXT: {context}
        QUESTION: {question}
    """)

    def concat_docs(docs) -> str: # retriever가 반환한 모든 Document들의 page_content를 하나의 단일 string으로 붙여주는 함수
        return "".join(doc.page_content for doc in docs)

    chain = {
        "context" : itemgetter('question') | retriever | concat_docs
        , "question" : itemgetter('question') | RunnablePassthrough()
    } | prompt| model | StrOutputParser()

    return chain

def get_answer(chain, question,  temp = .1, max_tokens = 2048):
    for model_name in ['claude3_haiku', 'claude3_5_sonnet', 'gpt3_5', 'gpt4o']:
        try:
            start = time()
            result = chain.with_config(configurable={
                        "llm": model_name, 
                        "temparature": temp,
                        "max_tokens": max_tokens
                    }).invoke({"question": question})
            print("\n\nModel: {} \t\t elapsed_time: {} \n\n Answer: {}".format(model_name, round(time() - start, 1), result))
            print("\n====================================================================================================")
        except:
            print("\n\nModel: {} \t\t Answer: {}".format(model_name, "Error."))
            print("\n====================================================================================================")

>  LLM 모델 정의, 나무위키 문서 데이터 Loading

In [4]:
llm = (
    ChatAnthropic(model_name='claude-3-5-sonnet-20240620')
    .configurable_alternatives(
        ConfigurableField(
            id = 'llm',
            name="LLM Model",
            description="The base LLM model",
        ),
        default_key="claude3_5_sonnet",
        claude3_haiku=ChatAnthropic(model_name='claude-3-haiku-20240307'),
        gpt4o = ChatOpenAI(model = 'gpt-4o'),
        gpt3_5 = ChatOpenAI(model = 'gpt-3.5-turbo'),
    )
    .configurable_fields(
        temperature=ConfigurableField(
            id="temperature",
            name="LLM Temperature",
            description="The temperature of the LLM",
        ),
        max_tokens = ConfigurableField(
            id="max_token",
            name="Maximum input Tokens",
            description="Maximum limit of input Tokens",
        ),
    )
)

url = 'https://namu.wiki/w/ILLIT'
docs = get_documnet_from_namuwiki(url = url , NamuLoader_obj = NamuLoader, hop = 1)


Document of ILLIT


================== Table of Contents  ==================

0. PROFILE
1. 개요
2. 멤버
	2.1. 멤버 간 케미
	2.2. 알유넥스트에서의 멤버들
	2.3. 알유넥스트의 멤버별 평가 내역
3. 특징
	3.1. 로고
	3.2. 그룹명
	3.3. 콘셉트
	3.4. 실력
		3.4.1. 보컬
		3.4.2. 댄스
		3.4.3. 랩
	3.5. 비주얼
	3.6. 인기 및 기록
	3.7. 세계관
4. 음반
5. 뮤직비디오
6. 활동
	6.1. 음악 방송 직캠
	6.2. 콘텐츠
		6.2.1. 숏폼
		6.2.2. Weverse
	6.3. 공연 및 행사
	6.4. 광고 및 화보
7. 수상 경력
8. 빌보드
	8.1. 빌보드 Hot 100
	8.2. 빌보드 200
	8.3. 빌보드 글로벌 200
	8.4. 빌보드 글로벌 200 (미국 제외)
9. 팬덤
	9.1. 응원법
	9.2. 굿즈
10. 여담
11. 논란 및 사건 사고
12. 역대 프로필 사진
EOD. FOOTNOTES
Main Doc ILLIT                                              Start (hop : 0/1)
  Sub Doc ILLIT/멤버 간 케미                            Start (hop : 1/1)
  Sub Doc ILLIT/멤버 간 케미                            Done! (hop : 1/1, elapsed_time: 1.0 seconds)
  Sub Doc SUMMER MOON                              Start (hop : 1/1)
  Sub Doc SUMMER MOON                              Done! (hop : 1/1, elapsed_time: 0.6 seconds)
  Sub Doc ILLIT/음반                                

## 벡터 스토어 비교

In [5]:
k = 5 ## 반환할 최대 문서 개수

question1 = "음반에 대해 설명해줘"


### Vanilla FAISS 벡터 스토어

In [6]:
vs = FAISS.from_documents(docs, embedding = EMBEDDING_FUNC)
ret = vs.as_retriever(search_kwargs={'k': k})
chain = get_chain(ret, llm)
get_answer(chain, question1)



Model: claude3_haiku 		 elapsed_time: 2.5 

 Answer: 죄송합니다. 제가 찾은 정보로는 ILLIT의 음반에 대한 설명을 자세히 알려드릴 수 없습니다. 문서에서는 ILLIT의 공연, 행사, 멤버들의 케미, 데뷔 앨범 수록곡 중 "My World"등에 대한 내용은 있었지만, 구체적인 음반 정보는 없었습니다. 관련 추가 정보는 제공된 링크를 참고하시길 바랍니다.



Model: claude3_5_sonnet 		 elapsed_time: 3.7 

 Answer: 죄송합니다. 주어진 문맥에서 ILLIT의 음반에 대한 구체적인 정보를 찾을 수 없습니다. 다만 데뷔 앨범에 'My World'라는 수록곡이 있다는 것만 언급되어 있습니다. 음반에 대한 더 자세한 설명을 원하신다면 추가적인 정보가 필요할 것 같습니다.



Model: gpt3_5 		 elapsed_time: 2.4 

 Answer: 음반은 아티스트들의 음악과 콘텐츠를 수록한 앨범을 말합니다. ILLIT의 음반은 데뷔 앨범을 통해 팬들에게 다양한 음악과 콘텐츠를 제공하고 있습니다. 더 자세한 정보는 다음 문서를 참고해주세요: https://namu.wiki/w/%EB%AF%BC%ED%9D%AC%EC%A7%84-HYBE%20%EA%B0%84%20ADOR%20%EA%B2%BD%EC%98%81%EA%B6%8C%20%EB%B6%84%EC%9F%81.



Model: gpt4o 		 elapsed_time: 0.7 

 Answer: No Answer found.



##### 답변에 사용된 Document 보기

In [7]:
ret.get_relevant_documents(query=question1)

c:\Users\thdwo\Documents\Github\langchain_t\langchain_t\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='ILLIT의 공연 및 행사를 서술한 문서이다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EA%B3%B5%EC%97%B0%20%EB%B0%8F%20%ED%96%89%EC%82%AC', 'page_hop': 1, 'parent_page_index': '6.3.', 'parent_page_toc_item': '공연 및 행사', 'abs_page_toc_item': '활동/공연 및 행사//개요', 'index': '1.', 'toc_item': '개요', 'ancestor_toc_item': ''}),
 Document(page_content='ILLIT의 콘텐츠에 대해 정리한 문서.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%BD%98%ED%85%90%EC%B8%A0', 'page_hop': 1, 'parent_page_index': '6.2.', 'parent_page_toc_item': '콘텐츠', 'abs_page_toc_item': '활동/콘텐츠//개요', 'index': '1.', 'toc_item': '개요', 'ancestor_toc_item': ''}),
 Document(page_content='ILLIT의 멤버 간 케미를 설명한 문서.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_pa

### MetaFAISS 벡터 스토어
- 새롭게 정의한 FaissMetaVectorStore를 통해 생성한 메타 벡터스토어 리트리버를 활용한 결과
- 메타 데이터 중 `abs_page_toc_item`와 `base_page_url`만을 사용해 임베딩한 뒤 벡터스토어에 저장
- 입력 쿼리와 위의 `abs_page_toc_item`와 `base_page_url`만 비교해 CONTEXT Document들을 반환, 답변에 사용

In [8]:
vs_meta = FaissMetaVectorStore.from_documents(docs, embedding = EMBEDDING_FUNC, metadata_fields= ["abs_page_toc_item", "base_page_url"])
ret_meta = vs_meta.as_retriever(vectorStoreType='metadata', search_kwargs={'k': k})
chain_meta = get_chain(ret_meta, llm)
get_answer(chain_meta, question1)



Model: claude3_haiku 		 elapsed_time: 1.9 

 Answer: ILLIT이 발매한 음반에 대해 설명하면 다음과 같습니다:

SUPER REAL ME미니 1집이 2024년 3월 25일 발매되었습니다.

SUPER REAL ME (Sped Up)리믹스 EP가 2024년 4월 1일 발매되었습니다.

Magnetic (Remixes)리믹스 싱글이 2024년 4월 19일 발매되었습니다.

그 외에 추가적인 정보는 문서에 제공되어 있지 않습니다.



Model: claude3_5_sonnet 		 elapsed_time: 6.6 

 Answer: ILLIT이 발매한 음반에 대한 정보는 다음과 같습니다:

1. SUPER REAL ME: 2024년 3월 25일에 발매된 미니 1집입니다.

2. SUPER REAL ME (Sped Up): 2024년 4월 1일에 발매된 리믹스 EP입니다.

3. Magnetic (Remixes): 2024년 4월 19일에 발매된 리믹스 싱글입니다.

이 세 음반은 ILLIT의 초기 활동을 보여주는 작품들로, 정규 앨범과 리믹스 버전을 포함하고 있습니다. 특히 'SUPER REAL ME'는 그룹의 첫 번째 미니 앨범으로, 이후 스피드 업 버전과 리믹스 싱글이 연이어 발매되었습니다.



Model: gpt3_5 		 elapsed_time: 2.2 

 Answer: 걸그룹 ILLIT이 발매한 음반은 SUPER REAL ME 미니 1집이 2024년 3월 25일에 발매되었습니다. 또한 SUPER REAL ME (Sped Up) 리믹스 EP는 2024년 4월 1일에 발매되었고, Magnetic (Remixes) 리믹스 싱글은 2024년 4월 19일에 발매되었습니다.



Model: gpt4o 		 elapsed_time: 2.4 

 Answer: ILLIT이 발매한 음반에 대해 설명드리겠습니다.

1. **SUPER REAL ME**: ILLIT의 첫 번째 미니 앨범으로 2024년 3월 25일에 발매되었

##### 답변에 사용된 Document 보기

In [9]:
ret_meta.get_relevant_documents(question1)

[Document(page_content='걸그룹 ILLIT이 발매한 음반을 서술한 문서이다. ,,,,SUPER REAL ME미니 1집2024. 03. 25.,SUPER REAL ME (Sped Up)리믹스 EP2024. 04. 01.,Magnetic (Remixes)리믹스 싱글2024. 04. 19.,,', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//개요', 'index': '1.', 'toc_item': '개요', 'ancestor_toc_item': ''}),
 Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//국내 음반', 'index': '2.', 'toc_item': '국내 음반', 'ancestor_toc_item': ''}),
 Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https:

#### 두번째 질문

#### Vanilla FAISS 벡터 스토어

In [10]:
question2 = '연습생 생활이 얼마나 길었어?'
get_answer(chain, question2)



Model: claude3_haiku 		 elapsed_time: 2.2 

 Answer: 문서에 따르면 ILLIT 구성원 중 모카와 이로하는 오랜 연습생 생활을 통해 수준급의 춤 실력을 갖추었다고 언급되어 있습니다. 또한 윤아와 민주도 긴 연습생 생활을 한 것으로 서술되어 있습니다. 하지만 구체적으로 연습생 생활 기간이 얼마나 되었는지에 대해서는 명확한 정보가 제공되지 않아서 정확한 답변을 드리기가 어렵습니다.



Model: claude3_5_sonnet 		 elapsed_time: 10.1 

 Answer: 주어진 문맥에서 연습생 생활의 정확한 기간은 언급되어 있지 않습니다. 하지만 다음과 같은 정보를 찾을 수 있습니다:

1. 윤아와 민주는 "긴 연습생 생활"을 했다고 언급되어 있습니다.
2. 원희는 "비교적 짧은 연습생 생활"을 했다고 나와 있습니다.

그러나 구체적인 기간은 명시되어 있지 않습니다. 다른 멤버들의 연습생 기간에 대한 정보는 제공되지 않았습니다.



Model: gpt3_5 		 elapsed_time: 1.6 

 Answer: 윤아와 민주는 긴 연습생 생활을 했으며, 원희는 상대적으로 짧은 연습생 생활을 했다.



Model: gpt4o 		 elapsed_time: 3.9 

 Answer: 윤아와 민주는 긴 연습생 생활을 했습니다.



##### 답변에 사용된 Document 보기

In [11]:
ret.get_relevant_documents(query=question2)

[Document(page_content='일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 간식 몰래 먹는 혈육 | 10분 논쟁 EP.03,, 일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 간식 몰래 먹는 혈육 | 10분 논쟁 EP.03,,', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%BD%98%ED%85%90%EC%B8%A0', 'page_hop': 1, 'parent_page_index': '6.2.', 'parent_page_toc_item': '콘텐츠', 'abs_page_toc_item': '활동/콘텐츠//공식채널/10분 논쟁', 'index': '3.5.', 'toc_item': '10분 논쟁', 'ancestor_toc_item': '공식채널'}),
 Document(page_content='공식적인 포지션은 없지만, 모카와 이로하가 주요 댄스 포지션으로 구분된다.  이로하: 만 3세에 전문적인 춤을 추는 영상이 있을 정도로 어릴 적부터 춤 연습을 해왔으며, 파워풀하면서 정확한 춤선이 특징이다.  모카: 과거 댄스 크루원으로 활동했던 만큼 춤 실력이 뛰어나며, 춤선이 깔끔하고 정확하다. 표정연기로도 호평을 받았다.  모카와 이로하 이

### MetaFAISS 벡터 스토어

In [12]:
get_answer(chain_meta, question2)



Model: claude3_haiku 		 elapsed_time: 1.5 

 Answer: 죄송합니다. 제공된 문서에는 연습생 생활에 대한 구체적인 정보가 포함되어 있지 않습니다. 따라서 이 질문에 대한 적절한 답변을 찾을 수 없습니다.



Model: claude3_5_sonnet 		 elapsed_time: 2.8 

 Answer: No Answer found.



Model: gpt3_5 		 elapsed_time: 1.1 

 Answer: No Answer found.



Model: gpt4o 		 elapsed_time: 0.8 

 Answer: No Answer found.



##### 답변에 사용된 Document 보기

In [13]:
ret_meta.get_relevant_documents(query=question2)

[Document(page_content="일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름, 일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름,", metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%ED%99%9C%EB%8F%99', 'page_hop': 1, 'parent_page_index': '6.', 'parent_page_toc_item': '활동', 'abs_page_toc_item': '활동//데뷔 전', 'index': '2.', 'toc_item': '데뷔 전', 'ancestor_toc_item': ''}),
 Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EA%B3%B5%EC%97%B0%20%EB%B0%8F%20%ED%96%89%EC%82%AC', 'page_hop': 1, 'parent_page_index': '6.3.', 'parent_page_toc_item': '공연 및 행사', 'abs_page_toc_item': '활동/공연 및 행사//쇼케이스'

### 두 벡터 스토어 Ensemble로 사용하기

In [14]:
from langchain.retrievers import EnsembleRetriever

ret_content = vs_meta.as_retriever(vectorStoreType='page_content', search_kwargs={'k': k})
ret_meta = vs_meta.as_retriever(vectorStoreType='metadata', search_kwargs={'k': k})

ret_ensemble = EnsembleRetriever(
    retrievers=[ret_meta, ret_content], weights=[0.5, 0.5]
)

chain_ensemble = get_chain(ret_ensemble, llm)
get_answer(chain_ensemble, question1)



Model: claude3_haiku 		 elapsed_time: 2.4 

 Answer: SUPER REAL ME
- 미니 1집으로 2024년 3월 25일 발매되었습니다.

SUPER REAL ME (Sped Up)
- 리믹스 EP로 2024년 4월 1일 발매되었습니다.

Magnetic (Remixes)
- 리믹스 싱글로 2024년 4월 19일 발매되었습니다.

위의 세 가지 음반에 대해 문서에서 확인할 수 있는 정보를 정리하였습니다. 그 외에 추가적인 정보는 문서에서 찾을 수 없었습니다.



Model: claude3_5_sonnet 		 elapsed_time: 5.8 

 Answer: ILLIT이 발매한 음반에 대한 정보는 다음과 같습니다:

1. SUPER REAL ME (미니 1집)
   발매일: 2024년 3월 25일

2. SUPER REAL ME (Sped Up) (리믹스 EP)
   발매일: 2024년 4월 1일

3. Magnetic (Remixes) (리믹스 싱글)
   발매일: 2024년 4월 19일

이 정보에 따르면, ILLIT은 데뷔 미니앨범과 두 개의 리믹스 앨범을 발매했습니다. 데뷔 앨범인 'SUPER REAL ME'가 발매된 지 일주일 후에 같은 앨범의 스피드 업 버전이 나왔고, 그로부터 약 3주 후에 'Magnetic'의 리믹스 버전이 싱글로 발매되었습니다.



Model: gpt3_5 		 elapsed_time: 3.5 

 Answer: 해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.걸그룹 ILLIT이 발매한 음반을 서술한 문서이다. SUPER REAL ME 미니 1집은 2024년 3월 25일에 발매되었고, SUPER REAL ME (Sped Up) 리믹스 EP는 2024년 4월 01일에 발매되었으며, Magnetic (Remixes) 리믹스 싱글은 2024년 4월 19일에 발매되었습니다. 자세한 내용은 아래 링크를 참고해주세요: https://namu.wiki/w/%EB%AF%BC%ED%9D%A

In [15]:
ret_ensemble.get_relevant_documents(query=question1)

[Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//국내 음반', 'index': '2.', 'toc_item': '국내 음반', 'ancestor_toc_item': ''}),
 Document(page_content='걸그룹 ILLIT이 발매한 음반을 서술한 문서이다. ,,,,SUPER REAL ME미니 1집2024. 03. 25.,SUPER REAL ME (Sped Up)리믹스 EP2024. 04. 01.,Magnetic (Remixes)리믹스 싱글2024. 04. 19.,,', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//개요', 'index': '1.', 'toc_item': '개요', 'ancestor_toc_item': ''}),
 Document(page_content='ILLIT의 공연 및 행사를 서술한 문서이다.', metadata={'base_page_url': 'https://namu

In [16]:
get_answer(chain_ensemble, question2)



Model: claude3_haiku 		 elapsed_time: 3.1 

 Answer: 이 문서에서는 ILLIT 멤버들의 연습생 생활에 대한 정보를 구체적으로 제공하고 있지 않습니다. 다만 모카와 이로하의 경우 어릴 때부터 춤 연습을 해왔으며, 윤아와 민주도 긴 연습생 생활을 했다는 내용이 있습니다. 하지만 정확한 연습생 기간이나 다른 멤버들의 연습생 생활에 대해서는 언급이 없습니다. 따라서 이 문서만으로는 ILLIT 멤버들의 연습생 생활이 얼마나 길었는지에 대한 답변을 제공하기 어려우므로 "No Answer found."라고 응답하는 것이 적절할 것 같습니다.



Model: claude3_5_sonnet 		 elapsed_time: 9.9 

 Answer: 제공된 정보에 따르면 구체적인 연습생 기간은 명시되어 있지 않습니다. 다만 다음과 같은 관련 정보가 있습니다:

윤아와 민주는 "긴 연습생 생활"을 했다고 언급되어 있습니다. 반면 원희는 "비교적 짧은 연습생 생활"을 했다고 합니다.

정확한 기간은 명시되어 있지 않아 "No Answer found."라고 할 수 있지만, 멤버들 간에 연습 기간의 차이가 있었다는 점은 알 수 있습니다.



Model: gpt3_5 		 elapsed_time: 3.0 

 Answer: 모카와 이로하 이외에도 윤아와 민주도 긴 연습생 생활을 한 만큼 수준급의 춤 실력을 지녔다.원희 또한 비교적 짧은 연습생 생활에도 눈에 띄는 성장을 이루었다는 평가를 받았었다. 연습생 생활은 각각에 따라 다르며, 자세한 정보는 해당 문서에서 확인할 수 있습니다.



Model: gpt4o 		 elapsed_time: 1.2 

 Answer: No Answer found.



In [17]:
ret_ensemble.get_relevant_documents(query=question2)

[Document(page_content="일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름, 일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름,", metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%ED%99%9C%EB%8F%99', 'page_hop': 1, 'parent_page_index': '6.', 'parent_page_toc_item': '활동', 'abs_page_toc_item': '활동//데뷔 전', 'index': '2.', 'toc_item': '데뷔 전', 'ancestor_toc_item': ''}),
 Document(page_content='일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 간식 몰래 먹는 혈육 | 10분 논쟁 EP.03,, 일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 

In [18]:
get_answer(chain_ensemble, "이로하에 대해 설명해줘")



Model: claude3_haiku 		 elapsed_time: 3.3 

 Answer: 네, ILLIT 멤버 이로하에 대해 알아보겠습니다.

ILLIT은 2024년 3월 25일에 데뷔한 5인조 다국적 걸그룹입니다. 멤버 중 이로하는 데뷔 당시 만 16세의 막내 멤버입니다. 이로하의 생월은 2월이며, 숙소 룸메이트는 윤아와 모카입니다.

ILLIT 멤버 중 이로하를 제외한 4명(윤아, 민주, 모카, 원희)은 남매 관계입니다. 하지만 이로하는 외동이라고 합니다.

이상으로 ILLIT 멤버 이로하에 대한 정보를 정리해 보았습니다. 더 자세한 내용이 필요하시다면 추가로 확인해 드리겠습니다.



Model: claude3_5_sonnet 		 elapsed_time: 10.6 

 Answer: 제공된 정보에 따르면 이로하에 대해 다음과 같은 사실을 알 수 있습니다:

1. 이로하는 ILLIT이라는 5인조 다국적 걸그룹의 멤버입니다.

2. 그녀는 일본인 멤버입니다. (그룹은 한국인 3명, 일본인 2명으로 구성됨)

3. 이로하의 나이는 데뷔 당시(2024년 3월 25일) 만 16세입니다.

4. 그녀의 혈액형은 A형입니다.

5. 이로하의 생일은 2월입니다.

6. 그녀는 숙소에서 혼자 방을 쓰고 있습니다.

7. 이로하는 외동딸입니다.

이 정보들은 이로하에 대한 기본적인 사실들을 제공하지만, 그녀의 성격이나 특기, 그룹 내에서의 역할 등에 대한 더 자세한 정보는 주어진 컨텍스트에 없습니다.



Model: gpt3_5 		 elapsed_time: 3.0 

 Answer: 이로하는 빌리프랩소속 걸그룹 ILLIT의 멤버 중 한 명으로, 16세의 나이를 가지고 있습니다. 그녀는 이로하라는 이름을 가지고 있으며, 숙소 룸메이트는 모카와 함께 지내고 있습니다. 이로하는 외동이며, ILLIT의 멤버들 중에서는 가장 어린 연습생으로 소속되어 있습니다. 현재 ILLIT의 공식 리더는 없으며, 멤버 간에는 책임감을 가지고 모두가 리더라는 마음으로 함께 활동하고 

In [19]:
get_answer(chain_ensemble, "이로하와 민주의 케미는 어때?")



Model: claude3_haiku 		 elapsed_time: 2.7 

 Answer: 이로하와 민주는 ILLIT 멤버 중에서 가장 눈물이 많은 멤버라는 공통점이 있습니다. 방송에서도 민주가 눈물을 많이 보이는 모습이 나왔고, 이로하도 눈물이 많은 멤버로 평가받았기 때문에 이들은 "울보즈"라는 조합명으로 불리게 되었습니다. 이처럼 두 멤버의 눈물이 많다는 공통점으로 인해 팬들 사이에서 좋은 케미를 보이고 있는 것으로 보입니다.



Model: claude3_5_sonnet 		 elapsed_time: 7.3 

 Answer: 이로하와 민주의 케미는 다음과 같은 특징이 있습니다:

1. 울보즈: 두 멤버 모두 눈물이 많아서 '울보즈'라는 조합명이 있습니다. 방송에서 우는 모습을 자주 보여줬습니다.

2. 슈크림즈: 둘 다 슈크림붕어빵을 좋아해서 붙여진 이름입니다. (하지만 나중에 모카도 슈크림붕어빵파임이 밝혀졌습니다.)

3. 호박아즈: '호'카조노 이로하 + '박'민주 + 노윤'아'를 합친 조합명입니다.

4. 초식즈: 민주는 토끼, 이로하는 아기병아리로 비유되어 초식 동물 조합으로 불립니다.

이처럼 이로하와 민주는 여러 가지 공통점과 특징을 바탕으로 다양한 조합명을 가지고 있어, 팬들에게 인기 있는 케미를 보여주고 있습니다.



Model: gpt3_5 		 elapsed_time: 2.1 

 Answer: 이로하와 민주의 케미는 '울보즈' 팀 내에서 가장 눈물이 많은 멤버라는 공통점이 있는데, 함께 활동하며 눈물을 흘리는 모습이 많이 보여주기 때문에 케미가 좋다고 할 수 있습니다.



Model: gpt4o 		 elapsed_time: 2.7 

 Answer: 이로하와 민주는 둘 다 눈물이 많은 멤버로, 울보즈라는 조합을 이루고 있습니다. 이로하는 "알유넥스트 최종화 내 친구를 소개합니다." 영상에서 민주와 원희에게 눈물이 많은 멤버로 뽑혔고, 민주는 "알유넥스트 4라운드"에서 워스트를 받자 속상함에 눈물을 흘리며 방송에서 우는 